<a href="https://colab.research.google.com/github/harishpilla/Research/blob/Research/BreastCancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
x = data['data']
y = data['target']
print("shape of x: {}\nshape of y: {}".format(x.shape,y.shape))

shape of x: (569, 30)
shape of y: (569,)


In [31]:
from sklearn.model_selection import train_test_split
 
# To calculate the accuracy score of the model
from sklearn.metrics import accuracy_score, confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 10)
from sklearn.svm import SVC
 
# Building a Support Vector Machine on train data
svc_model = SVC(C= .1, kernel='linear', gamma= 1)
svc_model.fit(X_train, y_train)
 
prediction = svc_model .predict(X_test)
# check the accuracy on the training set
print(svc_model.score(X_train, y_train))
print(svc_model.score(X_test, y_test))

0.9846153846153847
0.9736842105263158


In [3]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x = sc.fit_transform(x)

In [4]:
from torch.utils.data import Dataset, DataLoader
class dataset(Dataset):
  def __init__(self,x,y):
    self.x = torch.tensor(x,dtype=torch.float32)
    self.y = torch.tensor(y,dtype=torch.float32)
    self.length = self.x.shape[0]
 
  def __getitem__(self,idx):
    return self.x[idx],self.y[idx]
  def __len__(self):
    return self.length
trainset = dataset(x,y)
#DataLoader
trainloader = DataLoader(trainset,batch_size=64,shuffle=False)

In [27]:
from torch import nn
from torch.nn import functional as F
class Feedforward(nn.Module):
  def __init__(self,input_size):
    super(Feedforward,self).__init__()
    self.fc1 = nn.Linear(input_size,32)
    self.fc2 = nn.Linear(32,64)
    self.fc3 = nn.Linear(64,1)
  def forward(self,x):
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = torch.sigmoid(self.fc3(x))
    return x


class Feedforward_DLGN(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.input_size = input_size
        self.fc1_npf = nn.Linear(input_size,32)
        self.fc1_npv = nn.Linear(input_size,32)
        self.fc2_npf = nn.Linear(32,64)
        self.fc2_npv = nn.Linear(32,64)
        self.fc3_npf = nn.Linear(64,1)
        self.fc3_npv = nn.Linear(64,1)


    def forward(self,x,x_npv):
        
        Sigmoid=torch.nn.Sigmoid()
        beta=10
        self.Gating_value=[]

        output11 = self.fc1_npf(x)
        Gate11=Sigmoid(beta*output11)

        output12=self.fc2_npf(output11)
        Gate12=Sigmoid(beta*output12)

        output13=self.fc3_npf(output12)
        Gate13=Sigmoid(beta*output13)
        

        ############################################## NPV Starts ##############################################################  

        output = self.fc1_npv(x_npv)
        output=output*Gate11

        output=self.fc2_npv(output)
        output=output*Gate12

        output=self.fc3_npv(output)
        output=output*Gate13   

        output = torch.sigmoid(output)
        # output = self.softmax(output)
        
        return output

In [28]:
learning_rate = 0.01
epochs = 700
# Model , Optimizer, Loss
model = Feedforward(input_size=x.shape[1])
model_DLGN = Feedforward_DLGN(input_size=x.shape[1])
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)
loss_fn = nn.BCELoss()

In [29]:
losses = []
accur = []
for i in range(epochs):
  for j,(x_train,y_train) in enumerate(trainloader):
    
    #calculate output
    output = model(x_train)
 
    #calculate loss
    loss = loss_fn(output,y_train.reshape(-1,1))
 
    #accuracy
    predicted = model(torch.tensor(x,dtype=torch.float32))
    acc = (predicted.reshape(-1).detach().numpy().round() == y).mean()
    #backprop
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  if i%50 == 0:
    losses.append(loss)
    accur.append(acc)
    print("epoch {}\tloss : {}\t accuracy : {}".format(i,loss,acc))

epoch 0	loss : 0.7384915947914124	 accuracy : 0.37258347978910367
epoch 50	loss : 0.1793651133775711	 accuracy : 0.9490333919156415
epoch 100	loss : 0.09920590370893478	 accuracy : 0.9789103690685413
epoch 150	loss : 0.07948213070631027	 accuracy : 0.9789103690685413
epoch 200	loss : 0.06675739586353302	 accuracy : 0.9824253075571178
epoch 250	loss : 0.05708572641015053	 accuracy : 0.984182776801406
epoch 300	loss : 0.04892220348119736	 accuracy : 0.984182776801406
epoch 350	loss : 0.04291361942887306	 accuracy : 0.9876977152899824
epoch 400	loss : 0.03814249858260155	 accuracy : 0.9894551845342706
epoch 450	loss : 0.03444446623325348	 accuracy : 0.9912126537785588
epoch 500	loss : 0.03160341829061508	 accuracy : 0.9912126537785588
epoch 550	loss : 0.029017403721809387	 accuracy : 0.9912126537785588
epoch 600	loss : 0.027289019897580147	 accuracy : 0.9929701230228472
epoch 650	loss : 0.025856610387563705	 accuracy : 0.9929701230228472


In [30]:
losses = []
accur = []
for i in range(epochs):
  for j,(x_train,y_train) in enumerate(trainloader):
    
    #calculate output
    All_ones=torch.ones(x_train.shape)
    output = model_DLGN(x_train,All_ones)
 
    #calculate loss
    loss = loss_fn(output,y_train.reshape(-1,1))
 
    #accuracy
    predicted = model(torch.tensor(x,dtype=torch.float32))
    acc = (predicted.reshape(-1).detach().numpy().round() == y).mean()
    #backprop
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  if i%50 == 0:
    losses.append(loss)
    accur.append(acc)
    print("epoch {}\tloss : {}\t accuracy : {}".format(i,loss,acc))

epoch 0	loss : 0.6783239841461182	 accuracy : 0.9929701230228472
epoch 50	loss : 0.6783239841461182	 accuracy : 0.9929701230228472
epoch 100	loss : 0.6783239841461182	 accuracy : 0.9929701230228472
epoch 150	loss : 0.6783239841461182	 accuracy : 0.9929701230228472
epoch 200	loss : 0.6783239841461182	 accuracy : 0.9929701230228472
epoch 250	loss : 0.6783239841461182	 accuracy : 0.9929701230228472
epoch 300	loss : 0.6783239841461182	 accuracy : 0.9929701230228472
epoch 350	loss : 0.6783239841461182	 accuracy : 0.9929701230228472
epoch 400	loss : 0.6783239841461182	 accuracy : 0.9929701230228472
epoch 450	loss : 0.6783239841461182	 accuracy : 0.9929701230228472
epoch 500	loss : 0.6783239841461182	 accuracy : 0.9929701230228472
epoch 550	loss : 0.6783239841461182	 accuracy : 0.9929701230228472
epoch 600	loss : 0.6783239841461182	 accuracy : 0.9929701230228472
epoch 650	loss : 0.6783239841461182	 accuracy : 0.9929701230228472
